Importing Libs

In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score
from torch.optim.lr_scheduler import StepLR
from torch.cuda.amp import GradScaler, autocast

Initailizing Paths and vars, batch sizes and training data

In [ ]:
CKPT_PATH = 'model.pth.tar'
N_CLASSES = 14
CLASS_NAMES = ['Atelectasis', 'Cardiomegaly', 'Effusion', 'Infiltration', 'Mass', 'Nodule', 'Pneumonia',
               'Pneumothorax', 'Consolidation', 'Edema', 'Emphysema', 'Fibrosis', 'Pleural_Thickening', 'Hernia']
DATA_DIR = './Model-Ray02/images'
TEST_IMAGE_LIST = './Model-Ray02/labels/test_list.txt'
BATCH_SIZE = 64

Initialize mixed precision Training

In [ ]:
scaler = GradScaler()

Start training the Model

In [ ]:
def main():
    cudnn.benchmark = True

    # Initialize and load the model
    model = EfficientNetB0(N_CLASSES).cuda()
    model = torch.nn.DataParallel(model).cuda()

    if os.path.isfile(CKPT_PATH):
        print("=> loading checkpoint")
        checkpoint = torch.load(CKPT_PATH)
        model.load_state_dict(checkpoint['state_dict'])
        print("=> loaded checkpoint")
    else:
        print("=> no checkpoint found")

    # Data augmentation and normalization for training
    # Just normalization for validation
    normalize = transforms.Normalize([0.485, 0.456, 0.406],
                                     [0.229, 0.224, 0.225])
    test_dataset = ChestXrayDataSet(data_dir=DATA_DIR,
                                    image_list_file=TEST_IMAGE_LIST,
                                    transform=transforms.Compose([
                                        transforms.Resize(256),
                                        transforms.TenCrop(224),
                                        transforms.Lambda
                                        (lambda crops: torch.stack([transforms.ToTensor()(crop) for crop in crops])),
                                        transforms.Lambda
                                        (lambda crops: torch.stack([normalize(crop) for crop in crops]))
                                    ]))
    test_loader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE,
                             shuffle=False, num_workers=8, pin_memory=True)

    # Initialize the ground truth and output tensor
    gt = torch.FloatTensor()
    gt = gt.cuda()
    pred = torch.FloatTensor()
    pred = pred.cuda()

    # Switch to evaluate mode
    model.eval()
    for i, (inp, target) in enumerate(test_loader):
        target = target.cuda()
        gt = torch.cat((gt, target), 0)
        bs, n_crops, c, h, w = inp.size()
        input_var = torch.autograd.Variable(inp.view(-1, c, h, w).cuda(), volatile=True)
        with autocast():
            output = model(input_var)
        output_mean = output.view(bs, n_crops, -1).mean(1)
        pred = torch.cat((pred, output_mean.data), 0)

    AUROCs = compute_AUCs(gt, pred)
    AUROC_avg = np.array(AUROCs).mean()
    print('The average AUROC is {AUROC_avg:.3f}'.format(AUROC_avg=AUROC_avg))
    for i in range(N_CLASSES):
        print('The AUROC of {} is {}'.format(CLASS_NAMES[i], AUROCs[i]))

    # Compute additional metrics
    pred_labels = (pred > 0.5).float()
    f1 = f1_score(gt.cpu().numpy(), pred_labels.cpu().numpy(), average='macro')
    precision = precision_score(gt.cpu().numpy(), pred_labels.cpu().numpy(), average='macro')
    recall = recall_score(gt.cpu().numpy(), pred_labels.cpu().numpy(), average='macro')
    print('F1 Score: {:.3f}, Precision: {:.3f}, Recall: {:.3f}'.format(f1, precision, recall))

Func to Computes Area Under the Curve (AUC) from prediction scores

In [ ]:
def compute_AUCs(gt, pred):
    """Computes Area Under the Curve (AUC) from prediction scores.

    Args:
        gt: Pytorch tensor on GPU, shape = [n_samples, n_classes]
          true binary labels.
        pred: Pytorch tensor on GPU, shape = [n_samples, n_classes]
          can either be probability estimates of the positive class,
          confidence values, or binary decisions.

    Returns:
        List of AUROCs of all classes.
    """
    AUROCs = []
    gt_np = gt.cpu().numpy()
    pred_np = pred.cpu().numpy()
    for i in range(N_CLASSES):
        AUROCs.append(roc_auc_score(gt_np[:, i], pred_np[:, i]))
    return AUROCs

Func for returning transformed test Images

In [ ]:
def get_test_transform():
    normalize = transforms.Normalize([0.485, 0.456, 0.406],
                                     [0.229, 0.224, 0.225])
    return transforms.Compose([
        transforms.Resize(256),
        transforms.TenCrop(224),
        transforms.Lambda(lambda crops: torch.stack([transforms.ToTensor()(crop) for crop in crops])),
        transforms.Lambda(lambda crops: torch.stack([normalize(crop) for crop in crops]))
    ])

EfficientNet Model for Accuracy 

In [ ]:
class EfficientNetB0(nn.Module):
    def __init__(self, out_size):
        super(EfficientNetB0, self).__init__()
        self.efficientnet = torchvision.models.efficientnet_b0(pretrained=True)
        num_ftrs = self.efficientnet.classifier[1].in_features
        self.efficientnet.classifier = nn.Sequential(
            nn.Linear(num_ftrs, out_size),
            nn.Sigmoid()
        )
    def forward(self, x):
        x = self.efficientnet(x)
        return x


Loading the Model

In [ ]:
def load_model():
    model = EfficientNetB0(N_CLASSES).cuda()
    model = torch.nn.DataParallel(model).cuda()
    if os.path.isfile(CKPT_PATH):
        checkpoint = torch.load(CKPT_PATH)
        model.load_state_dict(checkpoint['state_dict'])
        print("Model loaded successfully.")
    else:
        raise FileNotFoundError("No checkpoint found at {}".format(CKPT_PATH))
    return model

Preprocessing the Images

In [ ]:
def preprocess_image(image_path):
    """Preprocess an image for prediction."""
    transform = get_test_transform()
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0)  # Add batch dimension
    return image.cuda()

Predicting phase

In [ ]:
def predict(image_path, model):
    model.eval()
    image = preprocess_image(image_path)
    with torch.no_grad():
        output = model(image)
    probabilities = output.cpu().numpy().flatten()
    return {CLASS_NAMES[i]: float(probabilities[i]) for i in range(N_CLASSES)}

Get responses

In [ ]:
def get_response(image_path):
    model = load_model()
    predictions = predict(image_path, model)
    response = {
        "predictions": predictions,
        "message": "Prediction completed successfully."
    }
    return response